In [1]:
import django, sys, os
sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from tmv_app.models import *
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from itertools import product, combinations
from utils.text import *
from pycountry_convert import  country_alpha2_to_continent_code, country_alpha3_to_country_alpha2
import scipy.stats as st
import pandas as pd

In [61]:
df = pd.read_csv('data/GAMI_Database_9.3.2020.csv', encoding="ISO-8859-1")
print(df.shape)
df.head()

(1682, 63)


,Article.ID,Summarize,1.1.Geography,GEOG_REGION,1.1.1.Geography-Country,1.2.Sector,1.3.Cross.cutting.topics,1.4.Indigenous.knowledge,1.5.Local.knowledge,2.1.1.Actors.Institutions,...,7.1.Methods,7.2.Coherence,7.3.Adequacy,7.4.Relevance,User.Note,Title,Journal,Authors,NumericScoreImplementationStage,Unnamed: 62
0,6205478,The study documents the effect of human capita...,Africa,Africa|,Bawku West district located in the north-easte...,Poverty livelihoods & sustainable development,Deserts/semi-arid areas/desertification,False,False,Individuals or households||| Local government,...,The method is robust enough as it combined nic...,"The article is coherent enough|||Yes, the arti...",The method is robust enough as it combined nic...,This paper is relevant to many African cases||...,|||,The response to climate variability among farm...,Agric. and Trade: Int. Perspect.,"Nti, F.K.; Barkley, A.",NaN,NaN
1,6205479,he aim of this study was to understand how fam...,Africa,Africa|,Tunisia,Food fibre & other ecosystem products|||Healt...,Biodiversity hotspots,True,True,Civil Society- sub-national or local||| Indiv...,...,|||Use of surveys,|||Yes,"|||Yes, the author shows the method used in co...","|||no, research findings can be applied in sim...","A complex paper, not clear in articulation and...",Sheep breeding in Central Tunisia: varied stra...,CAHIERS AGRICULTURES,"Jemaa, T; Huguenin, J; Moulin, CH; Najar, T",5.0,NaN
2,6205480,This study assessed the influence of climate v...,Africa,Africa|,Tanzania,Poverty livelihoods & sustainable development,None,False,False,Individuals or households,...,No|||Focus group discussions among men and women,No|||Somewhat - unclear of scales of change wh...,"No|||Sufficient: ""The FGDs approach was us...",Yes|||To studies on how climate variability af...,|||,I eat two meals per day impact of climate vari...,Agric. Food Secur.,"Saronga, N.J.; Mosha, I.H.; Kessy, A.T.; Ezeki...",1.0,NaN
3,6205481,the study evaluates cropping adaptation option...,Africa,Africa|,"Zimbabwe|||Samburu County, Kenya",Poverty livelihoods & sustainable development,None,False,False,Individuals or households,...,|||The research data was mostly quantitative. ...,"|||Yes, the article provides enough informatio...",|||Sample size of 500 is quite good (particula...,|||The findings from this study is relevant to...,|||,Migration and Self-Protection Against Climate ...,WORLD DEVELOPMENT,"Ng'ang'a, SK; Bulte, EH; Giller, KE; Mcintire,...",4.0,NaN
4,6205482,Using field-based data collected from two stud...,Africa,Africa|Europe,Mukonchi and Lunchu settlements in Central Pro...,"Health, well-being & communities",Deserts/semi-arid areas/desertification,False,False,Individuals or households,...,"Strong and coherent research approach, though ...",Sufficent to address most of coding questions|...,Coherent data|||Adequate.,Relevant for most African countries|||The find...,|||,Migration as an adaptive strategy to climate v...,DISASTERS,"Simatele, D; Simatele, M",NaN,NaN


In [62]:
for f in WoSArticle._meta.get_fields():
    if f._verbose_name not in df.columns:
        df[f._verbose_name] = None
    
df.head()

,Article.ID,Summarize,1.1.Geography,GEOG_REGION,1.1.1.Geography-Country,1.2.Sector,1.3.Cross.cutting.topics,1.4.Indigenous.knowledge,1.5.Local.knowledge,2.1.1.Actors.Institutions,...,Subject Category,Book Series Title,Special Issue,ISSN,Publication Name,Conference Sponsors,Supplement,Times Cited,Volume,Web of Science Category
0,6205478,The study documents the effect of human capita...,Africa,Africa|,Bawku West district located in the north-easte...,Poverty livelihoods & sustainable development,Deserts/semi-arid areas/desertification,False,False,Individuals or households||| Local government,...,None,None,None,None,None,None,None,None,None,None
1,6205479,he aim of this study was to understand how fam...,Africa,Africa|,Tunisia,Food fibre & other ecosystem products|||Healt...,Biodiversity hotspots,True,True,Civil Society- sub-national or local||| Indiv...,...,None,None,None,None,None,None,None,None,None,None
2,6205480,This study assessed the influence of climate v...,Africa,Africa|,Tanzania,Poverty livelihoods & sustainable development,None,False,False,Individuals or households,...,None,None,None,None,None,None,None,None,None,None
3,6205481,the study evaluates cropping adaptation option...,Africa,Africa|,"Zimbabwe|||Samburu County, Kenya",Poverty livelihoods & sustainable development,None,False,False,Individuals or households,...,None,None,None,None,None,None,None,None,None,None
4,6205482,Using field-based data collected from two stud...,Africa,Africa|Europe,Mukonchi and Lunchu settlements in Central Pro...,"Health, well-being & communities",Deserts/semi-arid areas/desertification,False,False,Individuals or households,...,None,None,None,None,None,None,None,None,None,None


In [115]:
for i,row in df.iterrows():
    t = row['Title'].replace('Ã\x82Â\xa0','').replace('Ã¢â\x82¬â\x80\x9c','').replace('Ã\x83Â¢','')
    try:
        d = Doc.objects.get(tslug=Doc.make_tslug(t))
    except:
        d = Doc.objects.filter(tslug=Doc.make_tslug(t)).first()
    if d is None:
        d = Doc.objects.filter(tslug__contains=Doc.make_tslug(t[:100])).first()
    if d is None:
        d = Doc.objects.filter(tslug__contains=Doc.make_tslug(t[:45])).first()
    if "hitherto neglected gender-sensitive public health perspective" in t:
        d = Doc.objects.filter(tslug__contains=Doc.make_tslug("hitherto neglected gender-sensitive public health perspective")).first()
    if "Seaweed (Mwani) farming" in t:
        d = Doc.objects.filter(tslug__contains=Doc.make_tslug("Seaweed (Mwani) farming")).first()
    #print(d.title)
    if d is None:
        print(t)
        continue
    for f in WoSArticle._meta.get_fields():
        if f.name in d.wosarticle.__dict__ and f.name!="cr":
            x = d.wosarticle.__dict__[f.name]
            if type(x) is list:
                x = "; ".join(x)
            df.loc[i,f._verbose_name] = x

Sistemas agroflorestais como estrategia de adaptaÃÂ§ÃÂ£o aos desafios das mudanÃÂ§as climÃÂ¡ticas no Brasil
PercepcÃÂ§aÃÆo de atores sociais de ItajaiÃÂ (SC) a respeito das variacÃÂ§oÃÆes climaÃÂticas, com foco nas inundacÃÂ§oÃÆes.pdf


In [117]:
df.to_csv('data/GAMI_Database_with_bibinfo.csv',index=False)

In [112]:
d = Doc.objects.filter(tslug__contains=Doc.make_tslug(t[:45])).first()
d

In [114]:
t[:45]

'Sistemas agroflorestais como estrategia de ad'

In [108]:
d.first().title

AttributeError: 'NoneType' object has no attribute 'first'

In [110]:
t = row['Title'].replace('Ã\x82Â\xa0','').replace('Ã\x83Â¢','')
t

'Sistemas agroflorestais como estrategia de adaptaÃ\x83Â§Ã\x83Â£o aos desafios das mudanÃ\x83Â§as climÃ\x83Â¡ticas no Brasil'

In [50]:
Doc.objects.filter(
    tslug__icontains=Doc.make_tslug("Adoption of Climate Smart Agricultural Technologies and Practices in Drylands in Uganda: Evidence from")
)

<QuerySet [<Doc: 2-s2.0-85071489232>]>

In [46]:
Doc.objects.filter(tslug=Doc.make_tslug(row['Title'])).values('title','PY','authors')

<QuerySet [{'title': 'Climate Change and Rural Development in the Middle Atlas Mountains and Fringe Areas (Morocco)', 'PY': 2016, 'authors': 'El Jihad, MD'}, {'title': 'Climate Change and Rural Development in the Middle Atlas Mountains and Fringe Areas (Morocco)', 'PY': 2016, 'authors': 'El Jihad, MD'}]>